In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Importing data and creating dataframes

In [71]:
data_folder = r'C:\Users\Cyrill\Documents\ada\project\project data\extracted data'

character_file = data_folder + '/character.metadata.tsv'
movie_file = data_folder + '/movie.metadata.tsv'
name_clusters_file = data_folder + '/name.clusters.txt' 
summaries_file = data_folder + '/plot_summaries.txt'
tvtropes_clusters_file = data_folder + '/tvtropes.clusters.txt'

In [72]:
column_names = [
    "wiki_movie_id",
    "freebase_movie_id",
    "release_date",
    "character_name",
    "actor_birth_date",
    "actor_gender",
    "actor_height_m",
    "actor_ethnicity_id",
    "actor_name",
    "actor_age_at_release",
    "character_actor_map_id",
    "character_id",
    "actor_id"
]
character = pd.read_csv(character_file, delimiter='\t', names = column_names)
character.head()

,wiki_movie_id,freebase_movie_id,release_date,character_name,actor_birth_date,actor_gender,actor_height_m,actor_ethnicity_id,actor_name,actor_age_at_release,character_actor_map_id,character_id,actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [73]:
column_names = ['Wikipedia_movie_ID',
                'Freebase_movie_ID',
                'Movie_name',
                'Movie_release_date',
                'Movie_box_office_revenu', 'Movie_runtime', 'Movie_languages', 'Movie_countries', 'Movie_genres']
movie = pd.read_csv(movie_file, delimiter='\t', names = column_names)
movie['Movie_release_date_datetime']= pd.to_datetime(movie['Movie_release_date'], format='mixed', errors='coerce')
movie['Movie_release_year'] = movie['Movie_release_date_datetime'].dt.strftime('%Y')
movie.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenu,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Movie_release_date_datetime,Movie_release_year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001-08-24,2001
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000-02-16,2000
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988-01-01,1988
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987-01-01,1987
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983-01-01,1983


### Finding all unique genres

In [78]:
movie['number_genres'] = movie['Movie_genres'].str.split(',',expand=True).count(axis='columns')
movie_genre = movie[['Wikipedia_movie_ID','number_genres']]
nb_movies = movie_genre['Wikipedia_movie_ID'].unique().size
genres = movie['Movie_genres'].str.split(r'{|}',expand=True).fillna(0)
genres = genres[1].str.split(',',expand=True).fillna(0)
L=[]

for i in range(nb_movies):
    L+=[[movie_genre['Wikipedia_movie_ID'][i]]]*movie_genre['number_genres'][i]

j=0
k=0
for i in range(nb_movies):
    while j<17 and genres[j][i]!=0: #get rid of 17
        L[k+j]=L[k+j]+[genres[j][i]]
        j+=1
    k=k+j
    j=0


df_genres = pd.DataFrame(L,columns=['Wikipedia_movie_ID','Movie_genres'])
df_genres2 = df_genres["Movie_genres"].str.split(r":",expand=True)
df_genres = df_genres.merge(df_genres2,left_index=True,right_index=True)

df_genres = df_genres.drop(columns=["Movie_genres"]).rename(columns={0: "Movie_genres_ID", 1: "Movie_genres"})
df_genres.columns
df_genres.head(30)

genres = df_genres['Movie_genres'].unique()

### Pirate work facilitator

In [79]:
# for quick testing
# genres = [' "Feature film"', ' "Neo-noir"', ' "Private military company"']

In [80]:
user_inputs = []

# Loop through each genre and prompt user for input
for i in range(len(genres)):
    previous_genre = 10
    this_genre = input(f"Input for {genres[i]}: ")

    # in case you want to correct the previous input, type '2'    
    if this_genre== '2' and user_inputs:
        # Correct the previous input
        previous_genre = input(f"Enter the correct input for {genres[i-1]}: ")
        this_genre = input(f"Input for {genres[i]}: ")
        
    # Validate user input
    while this_genre not in ['-1', '0', '1', '2'] or previous_genre not in ['-1', '0', '1', 10]:
        print("Invalid inputs. Please enter -1, 0, or 1. Enter 2 if you want to correct previous input.")
        this_genre = input(f"Input for {genres[i]}: (-1, 0, 1, 2) ")

        if this_genre== '2' and user_inputs:
        # Correct the previous input
            previous_genre = input(f"Enter the correct input for {genres[i-1]}: (-1, 0, 1) ")
            this_genre = input(f"Input for {genres[i]}: ")
    
    # Append the user input to the list
    if previous_genre != 10:
        user_inputs[i-1] = int(previous_genre)
    user_inputs.append(int(this_genre))

Input for  "Thriller":  0
Input for  "Science Fiction":  1
Input for  "Horror":  0
Input for  "Adventure":  0
Input for  "Supernatural":  1
Input for  "Action":  0
Input for  "Space western":  1
Input for  "Mystery":  0
Input for  "Biographical film":  -1
Input for  "Drama":  0
Input for  "Crime Drama":  0
Input for  "Crime Fiction":  1
Input for  "Erotic thriller":  0
Input for  "Psychological thriller":  0
Input for  "Short Film":  0
Input for  "Silent film":  0
Input for  "Indie":  0
Input for  "Black-and-white":  0
Input for  "Comedy":  0
Input for  "Family Film":  0
Input for  "Fantasy":  1
Input for  "World cinema":  0
Input for  "Musical":  0
Input for  "Japanese Movies":  0
Input for  "Action/Adventure":  0
Input for  "Romantic comedy":  0
Input for  "Ensemble Film":  0
Input for  "Comedy-drama":  0
Input for  "Romantic drama":  0
Input for  "Romance Film":  0
Input for  "Costume drama":  0
Input for  "War film":  0
Input for  "Epic":  0
Input for  "Period piece":  0
Input for 

Invalid inputs. Please enter -1, 0, or 1. Enter 2 if you want to correct previous input.


Input for  "Film adaptation": (-1, 0, 1, 2)  0
Input for  "Animation":  0
Input for  "Children's/Family":  0
Input for  "Children's Fantasy":  1
Input for None:  0
Input for  "Future noir":  1
Input for  "Comedy film":  0
Input for  "Coming of age":  0
Input for  "Suspense":  0
Input for  "Crime Thriller":  0
Input for  "Black comedy":  0
Input for  "Bollywood":  0
Input for  "Marriage Drama":  0
Input for  "Wuxia":  1
Input for  "Martial Arts Film":  0
Input for  "Chinese Movies":  0
Input for  "Western":  0
Input for  "Parody":  0
Input for  "Mockumentary":  0
Input for  "Fan film":  0
Input for  "Cult":  0
Input for  "Adventure Comedy":  0
Input for  "Slapstick":  0
Input for  "Biopic [feature]":  0
Input for  "Historical Epic":  0
Input for  "Sports":  0
Input for  "Family Drama":  0
Input for  "Political drama":  0
Input for  "Historical fiction":  1
Input for  "Courtroom Drama":  0
Input for  "Culture & Society":  0
Input for  "Biography":  -1
Input for  "Documentary":  -1
Input 

Invalid inputs. Please enter -1, 0, or 1. Enter 2 if you want to correct previous input.


Input for  "Teen": (-1, 0, 1, 2)  2
Enter the correct input for  "Docudrama": (-1, 0, 1)  -1
Input for  "Teen":  0
Input for  "Gothic Film":  0
Input for  "Disaster":  0
Input for  "Art film":  0
Input for  "Domestic Comedy":  0
Input for  "Musical comedy":  0
Input for  "Family-Oriented Adventure":  0
Input for  "Escape Film":  0
Input for  "Prison":  0
Input for  "Dance":  0
Input for  "Superhero movie":  0
Input for  "Filipino Movies":  0
Input for  "Filipino":  0
Input for  "Spy":  0
Input for  "Combat Films":  0
Input for  "Fantasy Adventure":  1
Input for  "Political cinema":  0
Input for  "Comedy horror":  0
Input for  "C-Movie":  0
Input for  "Dystopia":  0
Input for  "Buddy film":  0
Input for  "Tragedy":  0
Input for  "Time travel":  1
Input for  "Sword and sorcery":  1
Input for  "Political satire":  0
Input for  "Gross-out film":  0
Input for  "Gross out":  0
Input for  "Doomsday film":  1
Input for  "Plague":  0
Input for  "Propaganda film":  0
Input for  "Screwball comedy

Invalid inputs. Please enter -1, 0, or 1. Enter 2 if you want to correct previous input.


Input for  "Rockumentary": (-1, 0, 1, 2)  -1
Input for  "Feminist Film":  0
Input for  "Educational":  0
Input for  "Buddy cop":  0
Input for  "Film \u00e0 clef":  0
Input for  "Costume Horror":  0
Input for  "Pre-Code":  0
Input for  "Juvenile Delinquency Film":  0
Input for  "Christian film":  0
Input for  "Natural disaster":  0
Input for  "Movie serial":  0
Input for  "Linguistics":  0
Input for  "Language & Literature":  0
Input for  "Comedy of manners":  0
Input for  "Children's":  0
Input for  "Remake":  0
Input for  "Media Satire":  0
Input for  "Computer Animation":  0
Input for  "Spaghetti Western":  0
Input for  "Dogme 95":  0
Input for  "Backstage Musical":  0
Input for  "Law & Crime":  0
Input for  "Hybrid Western":  0
Input for  "Stop motion":  0
Input for  "Roadshow theatrical release":  0
Input for  "Sex comedy":  0
Input for  "Tollywood":  0
Input for  "Women in prison films":  0
Input for  "Holiday Film":  0
Input for  "Heavenly Comedy":  0
Input for  "Road movie":  0


Invalid inputs. Please enter -1, 0, or 1. Enter 2 if you want to correct previous input.


Input for  "Inventions & Innovations": (-1, 0, 1, 2)  0
Input for  "Instrumental Music":  0
Input for  "Media Studies":  0
Input for  "Archives and records":  0
Input for  "Psycho-biddy":  -1
Input for  "Outlaw biker film":  2
Enter the correct input for  "Psycho-biddy":  0
Input for  "Outlaw biker film":  0
Input for  "Prison escape":  0
Input for  "Z movie":  0
Input for  "News":  0
Input for  "Beach Film":  0
Input for  "Live action":  0
Input for  "Fictional film":  0
Input for  "Education":  0
Input for  "Health & Fitness":  0
Input for  "Film-Opera":  0
Input for  "Period Horror":  0
Input for  "Comdedy":  0
Input for  "Children's Issues":  0
Input for  "Science fiction Western":  1
Input for  "Bloopers & Candid Camera":  0
Input for  "Psychological horror":  0
Input for  "Albino bias":  0
Input for  "Family & Personal Relationships":  0
Input for  "Steampunk":  1
Input for  "Fantasy Drama":  1
Input for  "Sci-Fi Thriller":  1
Input for  "Courtroom Comedy":  0
Input for  "Operett

In [81]:
user_name = input('What is your name? for the file name of your inputs...')

What is your name? for the file name of your inputs... Cyrill


In [82]:
filename = user_name + '_genres.txt'

with open(filename, 'w') as file:
    for genre, user_input in zip(genres, user_inputs):
        file.write(f"{genre}: {user_input}\n")

print("User inputs saved to user_inputs.txt.")

User inputs saved to user_inputs.txt.
